In [ ]:
# Silver Layer - Github Repositories
# Process:
# 1. Import Libraries needed
# 2. Connect Bronze to Silver
# 3. For Validations
# - Remove Duplicate Repositories
# - Remove rows with null names
# - Standardize date formats
# 4. Transformations
# - Clean languages names
# - Remove special characters from descriptions
# - Convert dates to standard format
# 5. Quality flags
# - Mark repositories with missing languages
# - Mark repositories with missing description

In [1]:
import os
import json
import pandas as pd
from datetime import datetime
import pyarrow

In [2]:
BRONZE_FOLDER = "C:/Users/rjaya/OneDrive/Desktop/hop_dataeng/data/bronze"
SILVER_FOLDER = "C:/Users/rjaya/OneDrive/Desktop/hop_dataeng/data/silver"
COLLECTION_DATE = datetime.now().strftime("%Y-%m-%d")

In [3]:
bronze_json_files = sorted([filename for filename in os.listdir(BRONZE_FOLDER) if filename.endswith('.json')])
load_bronze_file_path = f"{BRONZE_FOLDER}/{bronze_json_files[-1]}"

In [4]:
with open(load_bronze_file_path) as file:
    data = json.load(file)

print(type(data))
print(len(data))
print(data[0])

flatten_repos = []

for i, item in enumerate(flatten_repos):
    if isinstance(item, str):
        print(f"Index {i}: {item}")

print(len(flatten_repos))
print(sum(1 for item in flatten_repos if isinstance(item, dict)))
print(sum(1 for item in flatten_repos if isinstance(item, str)))

<class 'list'>
51
[{'id': 940929652, 'node_id': 'R_kgDOOBVydA', 'name': '1590A', 'full_name': 'torvalds/1590A', 'private': False, 'owner': {'login': 'torvalds', 'id': 1024025, 'node_id': 'MDQ6VXNlcjEwMjQwMjU=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1024025?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/torvalds', 'html_url': 'https://github.com/torvalds', 'followers_url': 'https://api.github.com/users/torvalds/followers', 'following_url': 'https://api.github.com/users/torvalds/following{/other_user}', 'gists_url': 'https://api.github.com/users/torvalds/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/torvalds/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/torvalds/subscriptions', 'organizations_url': 'https://api.github.com/users/torvalds/orgs', 'repos_url': 'https://api.github.com/users/torvalds/repos', 'events_url': 'https://api.github.com/users/torvalds/events{/privacy}', 'received_events_url': 'https://api.g

In [5]:
flatten_repos = []
for user_repos in data:
    for repo in user_repos:
        flatten_repos.append(repo)


print(type(flatten_repos))
print(flatten_repos[0])

<class 'list'>
{'id': 940929652, 'node_id': 'R_kgDOOBVydA', 'name': '1590A', 'full_name': 'torvalds/1590A', 'private': False, 'owner': {'login': 'torvalds', 'id': 1024025, 'node_id': 'MDQ6VXNlcjEwMjQwMjU=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1024025?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/torvalds', 'html_url': 'https://github.com/torvalds', 'followers_url': 'https://api.github.com/users/torvalds/followers', 'following_url': 'https://api.github.com/users/torvalds/following{/other_user}', 'gists_url': 'https://api.github.com/users/torvalds/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/torvalds/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/torvalds/subscriptions', 'organizations_url': 'https://api.github.com/users/torvalds/orgs', 'repos_url': 'https://api.github.com/users/torvalds/repos', 'events_url': 'https://api.github.com/users/torvalds/events{/privacy}', 'received_events_url': 'https://api.githu

In [6]:
clean_repos = [repo for repo in flatten_repos if isinstance(repo, dict) and 'name' in repo]
df = pd.DataFrame(clean_repos)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           1270 non-null   int64 
 1   node_id                      1270 non-null   object
 2   name                         1270 non-null   object
 3   full_name                    1270 non-null   object
 4   private                      1270 non-null   bool  
 5   owner                        1270 non-null   object
 6   html_url                     1270 non-null   object
 7   description                  1120 non-null   object
 8   fork                         1270 non-null   bool  
 9   url                          1270 non-null   object
 10  forks_url                    1270 non-null   object
 11  keys_url                     1270 non-null   object
 12  collaborators_url            1270 non-null   object
 13  teams_url                    1270

In [7]:
df['owner_login'] = df['owner'].apply(lambda x: x['login'] if isinstance(x, dict) else None)
df_github_repos = df[['owner_login', 'name', 'description', 'language', 'stargazers_count', 'created_at', 'updated_at']].copy()

print("Shape:", df_github_repos.shape)
print("\nFirst few rows:")
print(df_github_repos.head())
print(df_github_repos.dtypes)


Shape: (1270, 7)

First few rows:
  owner_login            name  \
0    torvalds           1590A   
1    torvalds     GuitarPedal   
2    torvalds  libdc-for-dirk   
3    torvalds         libgit2   
4    torvalds           linux   

                                         description  language  \
0            Random odd guitar pedal design in kicad  OpenSCAD   
1                       Linus learns analog circuits  OpenSCAD   
2  Only use for syncing with Dirk, don't use for ...         C   
3  A cross-platform, linkable library implementat...         C   
4                           Linux kernel source tree         C   

   stargazers_count            created_at            updated_at  
0               437  2025-03-01T04:36:29Z  2025-11-07T15:08:41Z  
1              1167  2025-09-17T01:01:29Z  2025-11-14T21:52:13Z  
2               283  2017-01-17T00:25:49Z  2025-11-05T13:25:16Z  
3               233  2022-07-30T03:30:56Z  2025-11-14T05:55:17Z  
4            207061  2011-09-04T22:48:12

In [8]:
# Explorations
# Are there duplicate IDs?
duplicate_names = df_github_repos['name'].duplicated().sum()
print("Duplicate Names:", duplicate_names)

# Are there duplicate owner+name combinations?
duplicate_by_owner_name = df_github_repos[['owner_login', 'name']].duplicated().sum()
print("Duplicate owner+name:", duplicate_by_owner_name)

# Show me the actual duplicates
dup_mask = df_github_repos.duplicated(subset=['owner_login', 'name'], keep=False)
print(df_github_repos[dup_mask].sort_values(['owner_login', 'name']))

# Check the null values across all columns
print("Null values in each column:")
print(df_github_repos.isna().sum())

Duplicate Names: 43
Duplicate owner+name: 0
Empty DataFrame
Columns: [owner_login, name, description, language, stargazers_count, created_at, updated_at]
Index: []
Null values in each column:
owner_login           0
name                  0
description         150
language            331
stargazers_count      0
created_at            0
updated_at            0
dtype: int64


In [9]:
# Explorations
# Check the null values across all columns
print("Null values in each column:")
print(df_github_repos.isna().sum())

# Check for null names
null_names = df_github_repos['description'].isna().sum()
print(f"Null names: {null_names}")

# Show any null names rows
print(df_github_repos[df_github_repos['description'].isna()])

# Get complete daily quality report
print("Data Quality Report:")
print(df_github_repos.info())

Null values in each column:
owner_login           0
name                  0
description         150
language            331
stargazers_count      0
created_at            0
updated_at            0
dtype: int64
Null names: 150
     owner_login                                          name description  \
10     yyx990803                                    angular.js        None   
15     yyx990803  babel-plugin-transform-es2015-classes-simple        None   
17     yyx990803                         babel-preset-flow-vue        None   
26     yyx990803                         bundle-size-benchmark        None   
27     yyx990803                            bundle-vs-unbundle        None   
...          ...                                           ...         ...   
1250  shanselman                          aspneteverywheredemo        None   
1256  shanselman                                    azure-docs        None   
1257  shanselman                                  azure-friday        Non

#
#
#
#
#
##

In [10]:
df_github_repos[df_github_repos['language'] == 'Unknown'].shape[0]
print(df_github_repos[['language']])

        language
0       OpenSCAD
1       OpenSCAD
2              C
3              C
4              C
...          ...
1265          C#
1266        HTML
1267          C#
1268  JavaScript
1269        None

[1270 rows x 1 columns]


In [11]:
df_github_repos['language'] = df_github_repos['language'].fillna('Unknown')
print(df_github_repos['language'].count())

df_github_repos['description'] = df_github_repos['description'].fillna('No description provided')
print(df_github_repos['description'].count())

print((df_github_repos['language'] == 'Unknown').sum())
print((df_github_repos['description'] == 'No description provided').sum())

1270
1270
331
150


In [12]:
df_github_repos['created_at'] = pd.to_datetime(df_github_repos['created_at'])
df_github_repos['updated_at'] = pd.to_datetime(df_github_repos['updated_at'])

print(df_github_repos.dtypes)
print("\nFirst few rows after conversion:")
print(df_github_repos[["created_at", "updated_at"]].head())

owner_login                      object
name                             object
description                      object
language                         object
stargazers_count                  int64
created_at          datetime64[ns, UTC]
updated_at          datetime64[ns, UTC]
dtype: object

First few rows after conversion:
                 created_at                updated_at
0 2025-03-01 04:36:29+00:00 2025-11-07 15:08:41+00:00
1 2025-09-17 01:01:29+00:00 2025-11-14 21:52:13+00:00
2 2017-01-17 00:25:49+00:00 2025-11-05 13:25:16+00:00
3 2022-07-30 03:30:56+00:00 2025-11-14 05:55:17+00:00
4 2011-09-04 22:48:12+00:00 2025-11-15 00:04:33+00:00


In [13]:
# Create a Silver File Path to save Parquet Files
silver_file_path = f"{SILVER_FOLDER}/{COLLECTION_DATE}_github_users.parquet"

df_github_repos.to_parquet(silver_file_path)
print(f"Data saved to:", silver_file_path)


Data saved to: C:/Users/rjaya/OneDrive/Desktop/hop_dataeng/data/silver/2025-11-19_github_users.parquet
